In [68]:
#Credits to: ashwinmj

# Preamble
import string
import numpy as np

In [69]:
# Path of the text file containing the training data
training_data_file = 'a-million-dreams.txt'

In [70]:
#Helper functions
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation)) 
#empty char to empty char, and all punctuations are mapped to None

In [71]:
def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [72]:
def list2probabilitydict(given_list):
    probability_dict = {}
    given_list_length = len(given_list)
    for item in given_list:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        probability_dict[key] = value / given_list_length
    return probability_dict

In [73]:
initial_word = {}
second_word = {}
transitions = {}

In [74]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [75]:
# Trains a Markov model based on the data in training_data_file
def train_markov_model():
    for line in open(training_data_file):
        tokens = remove_punctuation(line.rstrip().lower()).split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:
                initial_word[token] = initial_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == tokens_length - 1:
                    add2dict(transitions, (prev_token, token), 'END')
                if i == 1:
                    add2dict(second_word, prev_token, token)
                else:
                    prev_prev_token = tokens[i - 2]
                    add2dict(transitions, (prev_prev_token, prev_token), token)
    
    # Normalize the distributions
    initial_word_total = sum(initial_word.values())
    for key, value in initial_word.items():
        initial_word[key] = value / initial_word_total
        
    for prev_word, next_word_list in second_word.items():
        second_word[prev_word] = list2probabilitydict(next_word_list)
        
    for word_pair, next_word_list in transitions.items():
        transitions[word_pair] = list2probabilitydict(next_word_list)
    
    print('Training successful.')

In [76]:
train_markov_model()

Training successful.


In [77]:
def sample_word(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

In [78]:
#Testing functions
number_of_sentences = 20

def generate():
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(initial_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))



In [79]:
generate()

but say that youll bring me along
a vision of the one i see
however big however small
to the world were gonna make
every room inside is filled
a million dreams
i close my eyes to see
a vision of the one i see
i dont care so call me crazy
a million dreams is all its gonna take
the world could be
they can say ive lost my mind
a vision of the one i see
a million dreams a million dreams are keeping me awake
the brightest colors fill my head
a million dreams
they can say they can say it all
they can say it all sounds crazy
but say that youll bring me along
they can say they can say it all
